# Experiment 3. 

I am running the algorithms for project influenced expert network.

In [1]:
import preprocessing as ps
import numpy as np
import networkx as nx
import pickle
import matplotlib.pyplot as plt
import random
random.seed(42)
import csv

In [2]:
# create the project networks
list_1 = [('DM', 'T'), ('DB', 'T'), ('AI', 'T')]
list_2 = [('DM', 'T'), ('DB', 'T'), ('AI', 'T'), ('DM', 'DB'), ('DM', 'AI'), ('DB', 'AI')]
list_3 = [('DM', 'T'), ('DB', 'T'), ('AI', 'DB')]

star_proj = ps.createProjectNetwork(list_1)
full_proj = ps.createProjectNetwork(list_2)
chain_proj = ps.createProjectNetwork(list_3)

## Experiment 3

In this experiment, I am running the algorithms on the project influenced network.
1. I run on samples of the network. (10, 20, 30, 40, 50)
   - First using uniform random sampling.
2. I am using the average sum of adjacent weights as the team influence measure. 

### Uniform Random Sampling

In [ ]:
projects = [(star_proj, "Star-Structured Project"), (full_proj, "Fully-connected Structured Project"), (chain_proj, "Chain-Structured Project")]

for i in range(10, 51, 10):
    network_main = pickle.load(open(f"./networks/subnets/{i}_nodes.pkl", 'rb'))
    network_main = ps.add_weights(network_main.copy())
    print(f">>>>>>>>>>>>>>>>>> {i} Node network <<<<<<<<<<<<<<<<<\n")
    with open(f'./results/exp_3/{i}_node_network.csv', 'w', newline='') as file:
        writer = csv.writer(file)

        for project in projects:
            network = ps.remove_edges_based_on_project_network(network_main, project[0]).copy()
            print(f"*********** {project[1]} ***********")
            writer.writerow([project[1]])
            writer.writerow([])
            # Greedy
            print("\n--------      Using Greedy Strategy     -------\n")
            obj_max_1 = 0.0
            best_set_1 = set()
            for node in network.nodes():
                subset, comm_eff = ps.Greedy(network, project[0], node, beta=None)
                if comm_eff > obj_max_1:
                    obj_max_1 = comm_eff
                    best_set_1 = subset

            leaders_eff_1 = ps.sum_edge_weights(network.subgraph(best_set_1))
            print("Coordinators communication efficiency", leaders_eff_1)
            print(f"Objective value : {obj_max_1}")
            team_eff_1 = round(obj_max_1 - leaders_eff_1, 4)

            for node in best_set_1:
                print(f"Team :{network.nodes[node]['label']}, Node: {node}, Rank: {leaders_eff_1}")
            
            title_attributes = f"Node Ranking for {project[1]} Under Greedy Strategy"
            writer.writerow([title_attributes])
            writer.writerow(['Author', "Label", "Rank"])
            for node in best_set_1:
                print(f"Team :{network.nodes[node]['label']}, Node: {node}, Rank: {ps.get_node_rank(network, node)}")
                writer.writerow([node, network.nodes[node]['label'], ps.get_node_rank(network, node)])
            writer.writerow([])

            # Influence Greedy
            print("\n--------      Using influence Greedy Strategy     -------\n")
            obj_max_2 = 0.0
            best_set_2 = set()
            print("Influential First")
            influential_nodes = ps.get_top_ranked_node_each_group(network)
            for node in influential_nodes:
                subset, comm_eff = ps.Greedy(network, project[0], node, beta=None)
                if comm_eff > obj_max_2:
                    obj_max_2 = comm_eff
                    best_set_2 = subset

            leaders_eff_2 = ps.sum_edge_weights(network.subgraph(best_set_2))
            print("Coordinators communication efficiency", leaders_eff_2)
            print(f"Objective value : {obj_max_2}")
            team_eff_2 = obj_max_2 - leaders_eff_2

            writer.writerow([])
            title_attributes = f"Node Ranking for {project[1]} Under Influence Greedy Strategy"
            writer.writerow([title_attributes])
            writer.writerow(['Author', "Label", "Rank"])
            for node in best_set_2:
                print(f"Team :{network.nodes[node]['label']}, Node: {node}, Rank: {ps.get_node_rank(network, node)}")
                writer.writerow([node, network.nodes[node]['label'], ps.get_node_rank(network, node)])
            writer.writerow([])

            writer.writerow([])
            title_attributes = f"Node Ranking for {project[1]} Under Influence Greedy Strategy"
            writer.writerow([title_attributes])
            writer.writerow(['Algorithm', "Objective Function", "Leaders Efficiency", "Team efficiency"])
            writer.writerow(['Greedy', obj_max_1, leaders_eff_1, team_eff_1])
            writer.writerow(['Inf Greedy', obj_max_2, leaders_eff_2, team_eff_2])
            writer.writerow([])

            print("\n\n")

## Using Monte Carlo for Random

In [3]:
network_main = pickle.load(open("./networks/network.pkl", 'rb'))
nx.number_of_isolates(network_main)

0

In [4]:
nodes_T = [node for node in network_main.nodes if network_main.nodes[node]['label'] == 'DB']
net_T = network_main.subgraph(nodes_T)
nx.number_of_isolates(net_T)

0

In [5]:
netty = ps.create_subnet(network_main, 10)
nx.number_of_isolates(netty)

0

In [6]:
netty.number_of_nodes()

40

In [ ]:
labels = ["T", "AI", "DB", "DM"]  # Replace "T" with the desired label
for label in labels:
    # Count the number of nodes with the specified label attribute
    count = sum(1 for node, attr in netty.nodes(data=True) if attr['label'] == label)
    print(f"{label}: {count}")

In [9]:
import pickle

for i in range(5, 21, 5):
    net = ps.create_subnet(network_main, i)
    file_path = f"./networks/new_subnets/{i}_nodes.pkl"
    with open(file_path, 'wb') as file:
        pickle.dump(net, file)

In [10]:
import pickle

file_path = "./networks/new_subnets/10_nodes.pkl"

with open(file_path, 'rb') as file:
    network = pickle.load(file)

# Now you can use the 'network' variable which contains the loaded network

In [14]:
projects = [(star_proj, "Star-Structured Project"), (full_proj, "Fully-connected Structured Project"), (chain_proj, "Chain-Structured Project")]

for i in range(5, 21, 5):
    network_main = pickle.load(open(f"./networks/new_subnets/{i}_nodes.pkl", 'rb'))
    # network_main = ps.add_weights(network_main.copy())

    for project in projects:
        network = ps.remove_edges_based_on_project_network(network_main.copy(), project[0]).copy()
        average_weight = ps.randomMonteCarlo(network_main, 1000)

        print(f"{i}_node_network: {project[1]} : {average_weight}")

Using Random : 36.7
5_node_network: Star-Structured Project : 36.7
Using Random : 37.0
5_node_network: Fully-connected Structured Project : 37.0
Using Random : 37.37
5_node_network: Chain-Structured Project : 37.37
Using Random : 38.84
10_node_network: Star-Structured Project : 38.84
Using Random : 39.52
10_node_network: Fully-connected Structured Project : 39.52
Using Random : 39.3
10_node_network: Chain-Structured Project : 39.3
Using Random : 50.76
15_node_network: Star-Structured Project : 50.76
Using Random : 48.64
15_node_network: Fully-connected Structured Project : 48.64
Using Random : 47.99
15_node_network: Chain-Structured Project : 47.99
Using Random : 26.57
20_node_network: Star-Structured Project : 26.57
Using Random : 26.48
20_node_network: Fully-connected Structured Project : 26.48
Using Random : 26.09
20_node_network: Chain-Structured Project : 26.09


In [ ]:
# 5 nodes

species = ("Star", "Chain", "Full")
penguin_means = {
    'Random': (),
    'Inf-Greedy': (50.209, 50.209, 50.209),
    'Greedy': (80.824, 80.824, 80.824)
}

x = np.arange(len(species))  # the label locations
width = 0.25  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(layout='constrained')

for attribute, measurement in penguin_means.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, measurement, width, label=attribute)
    ax.bar_label(rects, padding=3)
    multiplier += 1

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Length (mm)')
ax.set_title('Selected Leaders without considering project structure for 10 node network')
ax.set_xticks(x + width, species)
ax.legend(loc='upper left', ncols=3)
ax.set_ylim(0, 120)

plt.show()

In [ ]:
# Star Plot

x = [10, 20, 30, 40, 50]
y_greedy = [36.96, 77.7188, 70.9197, 138.2486, 162.7254]
y_inf_greedy = [9.078, 20.8476, 21.9526, 73.7888, 16.4414]

plt.plot(x, y_greedy, '*-', label='Greedy')
plt.plot(x, y_inf_greedy, 'o-', label='Inf-Greedy')
plt.xlabel('Number of nodes')
plt.ylabel(r"$C_{\eta}$")
plt.xticks(range(10, 51, 10))
# plt.title('Objective Function for Star-Structured Project')
plt.legend()
plt.show()

In [ ]:
# Chain Plot

x = [10, 20, 30, 40, 50]
y_greedy = [36.73, 77.5038, 54.4915, 136.3134, 162.3294]
y_inf_greedy = [8.618, 20.6326, 21.1986, 21.7183, 16.0454]

plt.plot(x, y_greedy, '*-', label='Greedy')
plt.plot(x, y_inf_greedy, 'o-', label='Inf-Greedy')
plt.xlabel('Number of nodes')
plt.ylabel(r"$C_{\eta}$")
plt.xticks(range(10, 51, 10))
# plt.title('Objective Function for Chain-Structured Project')
plt.legend()
plt.show()

In [ ]:
# Fully Plot

x = [10, 20, 30, 40, 50]
y_greedy = [36.96, 77.7188, 70.9197, 138.2486, 162.7254]
y_inf_greedy = [9.078, 20.8476, 21.9526, 73.7888, 16.4414]

plt.plot(x, y_greedy, '*-', label='Greedy')
plt.plot(x, y_inf_greedy, 'o-', label='Inf-Greedy')
plt.xlabel('Number of nodes')
plt.ylabel(r"$C_{\eta}$")
plt.xticks(range(10, 51, 10))
# plt.title('Objective Function for Chain-Structured Project')
plt.legend()
plt.show()